## Part 1: Preprocessing

In [30]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [31]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [32]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]

In [33]:
# Create a list of at least 10 column names to use as X data
selected_columns = [
    "Age", "JobLevel", "HourlyRate", "DistanceFromHome", "Education",
    "PerformanceRating", "JobInvolvement", "EnvironmentSatisfaction",
    "JobSatisfaction", "YearsWithCurrManager"
]


# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
print(X_df.dtypes)


Age                        int64
JobLevel                   int64
HourlyRate                 int64
DistanceFromHome           int64
Education                  int64
PerformanceRating          int64
JobInvolvement             int64
EnvironmentSatisfaction    int64
JobSatisfaction            int64
YearsWithCurrManager       int64
dtype: object


In [34]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_df, y_df, test_size=0.2, random_state=42
)

In [35]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# Convert categorical columns in X_train and X_test to numeric using LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Initialize the encoder
encoder = LabelEncoder()

# Apply the encoder to each column that is categorical in X_train and X_test
for col in X_train.select_dtypes(include=["object"]).columns:
    X_train[col] = encoder.fit_transform(X_train[col])
    X_test[col] = encoder.transform(X_test[col])



In [36]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [37]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[["Department"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = department_encoder.transform(y_train[["Department"]])
y_test_department_encoded = department_encoder.transform(y_test[["Department"]])



In [38]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[["Attrition"]])

# Create two new variables by applying the encoder 
# to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train[["Attrition"]])
y_test_attrition_encoded = attrition_encoder.transform(y_test[["Attrition"]])

## Create, Compile, and Train the Model

In [39]:
from tensorflow.keras import layers

# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer_1 = layers.Dense(64, activation="relu")(input_layer)
shared_layer_2 = layers.Dense(32, activation="relu")(shared_layer_1)


In [40]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(16, activation="relu")(shared_layer_2)

# Create the output layer
department_output_layer = layers.Dense(3, activation="softmax", name="department_output")(department_hidden_layer)


In [41]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(16, activation="relu")(shared_layer_2)

# Create the output layer
attrition_output_layer = layers.Dense(1, activation="sigmoid", name="attrition_output")(attrition_hidden_layer)

In [42]:
# Create the model
from tensorflow.keras.models import Model

model = Model(
    inputs=input_layer,
    outputs=[department_output_layer, attrition_output_layer]
)

# Compile the model
model.compile(
    optimizer="adam",
    loss={
        "department_output": "categorical_crossentropy",
        "attrition_output": "binary_crossentropy"
    },
    metrics={
        "department_output": "accuracy",
        "attrition_output": "accuracy"
    }
)

# Summarize the model
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64)        │        704 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      2,080 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 16)        │        528 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 16)        │        528 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         51 │ dense_10[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 1)         │         17 │ dense_11[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,908 (15.27 KB)

 Trainable params: 3,908 (15.27 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
# Train the model
history = model.fit(
    X_train_scaled,
    {
        "department_output": y_train_department_encoded,
        "attrition_output": y_train_attrition_encoded
    },
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/100


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 2), output.shape=(None, 1)

In [ ]:
# Evaluate the model with the testing data
test_metrics = model.evaluate(
    X_test_scaled,
    {
        "department_output": y_test_department_encoded,
        "attrition_output": y_test_attrition_encoded
    },
    verbose=1
)

In [ ]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_metrics[3]:.4f}")
print(f"Attrition Accuracy: {test_metrics[4]:.4f}")

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. The value of assessing accuracy in the assignment is moot. Following the instructions by the letter creates poorly matched target and output shapes; thus, the assignment cannot continue past model training.
2. Department - softmax as it is  multiclass. Attrition - sigmoid as it is single class.
3. Use binary encoding for Attrition rather than OneHotEncoding, utilize techniques like dropout and L2 to reduce overfitting, combine columns like YearsAtCompany and JobLevel to create new features, hyperparameter tuning and experimentation, control for data imbalances.